# Benchmark Rank-1 Modifications

In [ ]:
import numpy as np
import scipy.linalg
import scipy.stats

import cholupdates
import cholupdates.utils

In [ ]:
N = 5000

A = cholupdates.utils.random_spd_matrix(N, fast=True)
L = scipy.linalg.cholesky(A, lower=True)

## Update $A^+ = A + v v^T$

In [ ]:
v_ud = scipy.stats.norm.rvs(scale=10.0, size=N)

### Naive Implementations

In [ ]:
%%timeit
np.linalg.cholesky(A + np.outer(v_ud, v_ud))

In [ ]:
%%timeit
scipy.linalg.cho_factor(A + np.outer(v_ud, v_ud), check_finite=False)

In [ ]:
%%timeit
cholupdates.rank_1.update(
    L, v_ud,
    check_diag=False,
    method="cho_factor",
    check_finite=False,
)

In [ ]:
%%timeit
L_T_aug = np.vstack((L.T, v_ud))
R_aug, = scipy.linalg.qr(L_T_aug, mode="r")
L_ud = R_aug[:-1, :].T

### Seeger Algorithm

In [ ]:
%%timeit
cholupdates.rank_1.update(
    L, v_ud,
    check_diag=False,
    method="seeger",
    impl="python",
)

In [ ]:
%%timeit
cholupdates.rank_1.update(
    L, v_ud,
    check_diag=False,
    method="seeger",
    impl="cython",
)

### Seeger Algorithm (in-place)

In [ ]:
%%timeit L_copy = L.copy(order="K"); v_ud_copy = v_ud.copy()
cholupdates.rank_1.update(
    L_copy, v_ud_copy,
    check_diag=False,
    overwrite_L=True,
    overwrite_v=True,
    method="seeger",
    impl="python",
)

In [ ]:
%%timeit L_copy = L.copy(order="K"); v_ud_copy = v_ud.copy()
cholupdates.rank_1.update(
    L_copy, v_ud_copy,
    check_diag=False,
    overwrite_L=True,
    overwrite_v=True,
    method="seeger",
    impl="cython",
)

## Downdate $A^- = A - v v^T$

In [ ]:
v_dd = cholupdates.utils.random_rank_1_downdate(L)

### Naive Implementations

In [ ]:
%%timeit
np.linalg.cholesky(A - np.outer(v_dd, v_dd))

In [ ]:
%%timeit
scipy.linalg.cho_factor(A - np.outer(v_dd, v_dd), check_finite=False)

In [ ]:
%%timeit
cholupdates.rank_1.downdate(
    L, v_dd,
    check_diag=False,
    method="cho_factor",
    check_finite=False,
)

### Seeger Algorithm

In [ ]:
%%timeit
cholupdates.rank_1.downdate(
    L, v_dd,
    check_diag=False,
    method="seeger",
    impl="python",
)

In [ ]:
%%timeit
cholupdates.rank_1.downdate(
    L, v_dd,
    check_diag=False,
    method="seeger",
    impl="cython",
)

### Seeger Algorithm (in-place)

In [ ]:
%%timeit L_copy = L.copy(order="K"); v_dd_copy = v_dd.copy()
cholupdates.rank_1.downdate(
    L_copy, v_dd_copy,
    check_diag=False,
    overwrite_L=True,
    overwrite_v=True,
    method="seeger",
    impl="python",
)

In [ ]:
%%timeit L_copy = L.copy(order="K"); v_dd_copy = v_dd.copy()
cholupdates.rank_1.downdate(
    L_copy, v_dd_copy,
    check_diag=False,
    overwrite_L=True,
    overwrite_v=True,
    method="seeger",
    impl="cython",
)